<h4>This dataset presents a list of the 100 most streamed songs on Spotify of all time</h4>
(This analysis dates from March 8th 2023)  


Dataset source: https://www.kaggle.com/datasets/amaanansari09/most-streamed-songs-all-time




<h4>Features description:</h4>

- <b>duration:</b> Duration of the song (minutes).


- <b>energy:</b> A perceptual measure of intensity and activity.


- <b>key:</b> The harmony key, being 0 = C, 1 = C#, ... , 11 = B


- <b>loudness:</b> The overall loudness of a track in decibels (dB).


- <b>mode:</b> 0 = Major Key; 1 = Minor Key;


- <b>speechiness:</b> ?


- <b>acousticness:</b> ?


- <b>instrumentalness:</b> ?


- <b>liveness:</b> ?


- <b>valence:</b> ?


- <b>tempo:</b> Rhythm of the song (beats/min or bpm)


- <b>danceability:</b> ?






<h3>Objectives and Key Insights</h3>

<h4> Answers to be found:</h4>

1. How many artists made it to the top 100 most streamed songs of all time on Spotify?

2. Who are the artists with more songs in the list

3. Who are the artists with more streams in the list

4. Is there a pattern in terms of "key" and "mode" for these top songs?

5. Do these songs gravitate around an "optimal duration"?

6. Is there a correlation between "energy" and "tempo"?

7. Do these songs gravitate around an "optimal duration"?

8. What are the songs with more streams per day?

9. What artist have the oldest songs in the list, and what are these songs?

10. How much revenue all 100 songs have made together?



<h4> Insights that cannot be leveraged with this dataset (and I am particularly interested in!!):</h4>

1. What are the chord progressions of each song? Is there a pattern among these top streamed ones?

2. What is the primary language the song is sung in? And how many words are there?

3. What are the song structures used by each song? (e.g. Intro, Verse, Bridge, Pre Chorus, Chorus, Otro, etc.)

4. What is the total number of streams these artists have now and before launching each of these songs?

5. How much revenue each song has made?


In [8]:
# Import everything you need
import os
import pandas as pd
import numpy as np 
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import date
from dateutil.parser import parse

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 10000)

In [9]:
# Set-up the input files with da data to be analyzed
features_file_path = f"../datasets/Spotify Songs/Features.csv"

streams_file_path = f"../datasets/Spotify Songs/Streams.csv"

In [10]:
# Do all transformations needed to your data and enable analysis
df_features = pd.read_csv(features_file_path)
df_streams = pd.read_csv(streams_file_path)

df_merge = pd.merge(left=df_streams, left_on="Song", right=df_features, right_on="name")

df_merge = df_merge[["Song", "Artist", "Streams (Billions)", "Release Date", "duration", "energy", "key", "mode", "loudness", "tempo"]]

todays_date = date(2023, 3, 8)
df_merge.insert(4, "Today", todays_date)
df_merge['Release Date'] = df_merge['Release Date'].astype('string')
df_merge['Release Date'] = df_merge['Release Date'].apply(lambda x: parse(x).date())

df_merge['Aging in Days'] = df_merge['Today'] - df_merge['Release Date']
df_merge['Aging in Days'] = df_merge['Aging in Days'].dt.total_seconds() / (60*60*24)

df_merge.head()

,Song,Artist,Streams (Billions),Release Date,Today,duration,energy,key,mode,loudness,tempo,Aging in Days
0,Blinding Lights,The Weeknd,3.449,2019-11-29,2023-03-08,3.33,0.730,1,1,-5.934,171.005,1195.0
1,Shape of You,Ed Sheeran,3.398,2017-01-06,2023-03-08,3.90,0.652,1,0,-3.183,95.977,2252.0
2,Dance Monkey,Tones And I,2.770,2019-05-10,2023-03-08,3.49,0.588,6,0,-6.400,98.027,1398.0
3,Someone You Loved,Lewis Capaldi,2.680,2018-11-08,2023-03-08,3.04,0.405,1,1,-5.679,109.891,1581.0
4,Rockstar,Post Malone featuring 21 Savage,2.620,2017-09-15,2023-03-08,3.64,0.520,5,0,-6.136,159.801,2000.0


TODO: Address the issue of the "featuring" artists (e.g. "Post Malone featuring 21 Savage" is a different singer when compared to "Post Malone"). Perhaps I should consider only the main singer...

TODO: I would like to see what does the average streams per day look like for them...

In [11]:
# Function to split "featuring" and "and" artists and concat
def splitArtist(dataframe, separator):
    aux = dataframe[dataframe['Artist'].str.contains(r'\b\s{0}\s\b'.format(separator), case=True)]
    aux = aux['Artist'].str.split(pat=r'\s{0}\s'.format(separator), n=1, expand=True)
    aux = pd.concat([aux[0], aux[1]], axis=0)
    return aux

In [12]:
# Generate all artists list
def findAllArtists(artist_objs_list, separator_word_list, sep):
    aux = pd.concat(artist_objs_list, axis=0)
    
    # This could be done in a single line, but this way has better readability
    regex_aux = r'\b\s{0}\s\b'
    sep_aux = sep.join(separator_word_list)
    filter_aux = aux.str.contains(regex_aux.format(sep_aux))
    aux = aux[~filter_aux]
    # Alternative:
    # aux = aux[~aux.str.contains(r'\b\s({0})\s\b'.format(sep.join(separator_word_list)))]
    
    aux = aux.dropna()
    aux = aux.drop_duplicates()
    return aux

In [13]:
feat_df = splitArtist(df_merge, 'featuring')
and_df = splitArtist(df_merge, 'and')

artist_objs_list = [feat_df, and_df, df_merge['Artist']]
separator_word_list = ['featuring', 'and']
separator = '|'

all_artists = findAllArtists(artist_objs_list=artist_objs_list,         separator_word_list=separator_word_list, sep=separator)

In [14]:
df_uniq_art = pd.DataFrame(all_artists).rename(columns={0:'Artist'}).sort_values(by='Artist').reset_index(drop=True)
df_uniq_art['Songs'] = None
df_uniq_art['Streams (Billions)'] = None
df_uniq_art['Streams per day (Millions)'] = None
df_uniq_art['Oldest song'] = None
df_uniq_art['Oldest song aging'] = None
df_uniq_art['Estimated Earnings (Millions USD)'] = None


# Count songs per artist
def countSongs(artistName, dataframeColumn):
    count = (dataframeColumn.str.contains(artistName)).sum()
    return count

for index, row in df_uniq_art.iterrows():
    row[1] = countSongs(artistName=row[0], dataframeColumn=df_merge['Artist'])
    
    
# Sum streams per artist
def sumStreams(dataframe, artistName):
    aux = dataframe.loc[dataframe['Artist'].str.contains(artistName)]
    totalStreams = np.round(aux['Streams (Billions)'].sum(), 2)
    return totalStreams

for index, row in df_uniq_art.iterrows():
    row[2] = sumStreams(dataframe=df_merge, artistName=row[0])


# Find oldest song's name and aging
def findAging(dataframe, artistName):
    aux = dataframe.loc[dataframe['Artist'].str.contains(artistName)]
    aux = aux.sort_values(by='Aging in Days', ascending=False)
    oldestSong = aux['Song'].values[0]
    oldestSongAging = aux['Aging in Days'].values[0]
    return oldestSong, oldestSongAging

for index, row in df_uniq_art.iterrows():
    df_uniq_art.loc[index, 'Oldest song'], df_uniq_art.loc[index, 'Oldest song aging'] = findAging(dataframe=df_merge, artistName=row[0])
    
    
# Count streams per day
def streamsPerDay(dataframe):
    for index, row in dataframe.iterrows():
        dataframe.loc[index, 'Streams per day (Millions)'] = np.round(((10**3) * row[2]/row[5]), 2)
    return dataframe

df_uniq_art = streamsPerDay(df_uniq_art)

# Compute estimated earnings, if 4000 USD per million views
def computeEarnings(dataframe):
    for index, row in dataframe.iterrows():
        dataframe.loc[index, "Estimated Earnings (Millions USD)"] = np.round((4000 * row[3]) * row[5]/(10**6), 2)
    return dataframe

df_uniq_art = computeEarnings(df_uniq_art)


# Estimate each song earnings
df_songs = df_merge[['Song', 'Streams (Billions)', 'Aging in Days']]
df_songs['Estimated Earnings (Million USD)'] = None

def songEarnings(dataframe):
    for index, row in dataframe.iterrows():
        dataframe.loc[index, 'Estimated Earnings (Million USD)'] = np.round((4000 * (10**-3) * row[1]), 2)
    return dataframe

df_songs_final = songEarnings(df_songs)
df_songs_final

/var/folders/tx/qf9y3mqn2w55cr5yxxdm0f340000gn/T/ipykernel_47107/1045820153.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_songs['Estimated Earnings (Million USD)'] = None


,Song,Streams (Billions),Aging in Days,Estimated Earnings (Million USD)
0,Blinding Lights,3.449,1195.0,13.8
1,Shape of You,3.398,2252.0,13.59
2,Dance Monkey,2.770,1398.0,11.08
3,Someone You Loved,2.680,1581.0,10.72
4,Rockstar,2.620,2000.0,10.48
5,Sunflower,2.575,1602.0,10.3
6,One Dance,2.556,2528.0,10.22
7,Closer,2.481,2413.0,9.92
8,Stay,2.432,607.0,9.73
9,Believer,2.410,2226.0,9.64
